In [ ]:
import os
import random
from os.path import join
from time import time

import numpy as np
import pandas as pd
import torch
from tensorflow.keras.utils import to_categorical
from torch import optim
from torch.nn import BCELoss
from torch.optim import Adam
from torch.utils.data import TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from models.de_HHAR import DAE
from models.dis_z import DIS_Z
from models.activity_recognition import *
from utils.function import *

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]=str(0); 

In [ ]:
class Args:
    def __init__(self):
        self.root = '../data'
        self.batchSize = 64
        self.maxEpochs = 200
        self.nz = 200
        self.lr = 1e-4
        self.fSize = 64
        self.outDir = 'data/experiments/DAE_EVAL_HHAR_Noise'
        self.commit = 'eval'
        self.alpha = 1.0
        self.M = 5
        self.loss = 'MSE' #'BCE'
        self.loadDAE = False
        self.loadSVM = False    
        self.load_DAE_from = None
        self.evalMode = False
        self.comment = ''
        self.momentum = 0.1
        self.c = 0.01
        self.svmLR = 1e-4
        self.Ntest = 100
        self.gpuNo = 0
        self.multimodalZ = False
        self.imSize = 64
        self.sigma = [0.1]
        self.train_split = 0.8
random_seed=0

In [ ]:
def build_dis(args, dae, multimodalZ):
    if not multimodalZ:
        print('\n ** USING NORMAL PRIOR **')
        prior = dae.norm_prior
        NZ = args.nz
    else:
        print('\n ** USING MULTIMODAL PRIOR **')
        prior = dae.multi_prior
        NZ = 2
    dis = DIS_Z(nz=NZ, prior=prior)

    return dis, NZ

In [ ]:
data_folder_path='../../../Dataset/'
X_total = np.load(data_folder_path + 'hhar_time_X.npy').astype('float32')
y_total = np.load(data_folder_path + 'hhar_time_y.npy').astype(int)

X_total=np.nan_to_num(X_total)
for i in range(X_total.shape[1]):
    ch_data = X_total[:,i,:] # the data of channel id
    scaler = MinMaxScaler()
    ch_data = scaler.fit_transform(ch_data) # scale the data in this channel to [0,1]
    X_total[:,i,:] = ch_data # assign normalized data to normalized_X     

X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=random_seed)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2])

In [ ]:
def prepare_dataloaders(args, X_train, X_test, y_train, y_test):
    trainDataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    testDataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    
    trainLoader = torch.utils.data.DataLoader(trainDataset,
        batch_size=args.batchSize, shuffle=True) 

    testLoader = torch.utils.data.DataLoader(testDataset,
        batch_size=args.batchSize, shuffle=False)
    return trainLoader, testLoader

In [ ]:
def train_dae(args, exDir, trainLoader, testLoader):
    dae = DAE(nz=args.nz, imSize=args.imSize, fSize=args.fSize, sigma=args.sigma, multimodalZ=args.multimodalZ) #sigma=level of corruption
    
    if dae.useCUDA:
        torch.cuda.set_device(args.gpuNo)
        dae.cuda()
    
    save_input_args(exDir, args)  #save training opts
    save_exp_details(args, exDir)

    # Create optimizers
    optimDAE = optim.RMSprop(dae.parameters(), lr = args.lr, momentum=args.momentum)

    # Keeping track of training
    losses = {'enc': [], 'rec': [], 'dis':[], 'test rec':[]}

    with torch.autograd.set_detect_anomaly(True):
        for e in range(args.maxEpochs):

            epochEncLoss=0
            epochRecLoss=0
            epochDisLoss=0
            epochKlLoss=0

            for i, data in enumerate(trainLoader):
                T = time()
                dae.train()
                x, y = prep_data(data, useCUDA=dae.useCUDA)
                zFake, xRec = dae.forward(x)
                recLoss = dae.rec_loss(xRec, x, loss=args.loss)  #loss='BCE' or 'MSE'
                optimDAE.zero_grad()
                recLoss.backward()
                optimDAE.step()            
                epochRecLoss+=recLoss.detach().cpu().numpy()
            
                if i%100 == 0:
                    print('[%d, %d] rec: %0.5f, time: %0.3f' % (e, i, recLoss.detach().cpu().numpy(), time() - T))                    

            losses['rec'].append(epochRecLoss/i)

            # Test
            # dae.eval()

            # Save parameters
            dae.save_params(exDir)

In [ ]:
# if __name__ == "__main__":
args = Args()
args.random_seed = random_seed
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

print(X_train.shape)
print(y_train.shape)
trainLoader, testLoader = prepare_dataloaders(args, X_train, X_test, y_train, y_test)

exDir = make_new_folder(args.outDir)
print('Outputs will be saved to:', exDir)

In [ ]:
train_dae(args, exDir, trainLoader, testLoader)